<h1 align = "center"> Spark RDD </h1>

Ověření inicializace Spark contextu.

In [1]:
sc.version

2.4.3

## Vytvoření RDD

Vytvoření RDD pomocí paralelizace kolekcí.

In [ ]:
val data = Array(1, 2, 3, 4, 5)
val distData = sc.parallelize(data)
distData.collect().foreach(println)

Vytvoření RDD z textového souboru pomocí Spark contextu.

In [4]:
val logFile = sc.textFile("/resources/Prezentace/LabData/notebook.log")
logFile.take(10).foreach(println)

[I 12:09:13.491 NotebookApp] Using MathJax: /static/vendor/MathJax-2.5-latest/MathJax.js
[I 12:09:13.494 NotebookApp] Using existing profile dir: u'/home/notebook/.ipython/profile_default'
[I 12:09:13.513 NotebookApp] Writing notebook server cookie secret to /home/notebook/.ipython/profile_default/security/notebook_cookie_secret
[C 12:09:13.565 NotebookApp] WARNING: The notebook server is listening on all IP addresses and not using encryption. This is not recommended.
[C 12:09:13.565 NotebookApp] WARNING: The notebook server is listening on all IP addresses and not using authentication. This is highly insecure and not recommended.
[I 12:09:13.586 NotebookApp] Serving notebooks from local directory: /resources
[I 12:09:13.586 NotebookApp] 0 active kernels 
[I 12:09:13.586 NotebookApp] The IPython Notebook is running at: http://[all ip addresses on your system]:8888/
[I 12:09:13.586 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[W 

logFile = /resources/Prezentace/LabData/notebook.log MapPartitionsRDD[3] at textFile at <console>:27


/resources/Prezentace/LabData/notebook.log MapPartitionsRDD[3] at textFile at <console>:27

Vytvoření nového RDD pomocí transformace existujícího.

In [ ]:
val error = logFile.filter(line => line.contains("ERROR"))
error.take(10).foreach(println)

In [ ]:
val timesThree = distData.map(x => x * 3)
timesThree.collect().foreach(println)

## Akce nad RDD

Zjištění počtu řádků v RDD.

In [ ]:
logFile.count()

Maximální hodnota v RDD.

In [ ]:
distData.max()

Minimální hodnota v RDD.

In [ ]:
distData.min()

Vypsání statistiky z RDD.

In [ ]:
distData.stats()

Vrácení všech elementů RDD do kolekce.

In [ ]:
distData.collect()

Vrácení určitého množství elementů RDD do kolekce.

In [ ]:
logFile.take(10)

Vrátit první řádek.

In [5]:
logFile.first()

[I 12:09:13.491 NotebookApp] Using MathJax: /static/vendor/MathJax-2.5-latest/MathJax.js

Sečtení RDD.

In [ ]:
distData.reduce((x, y) => x + y)

In [ ]:
distData.sum()

## Cache

In [5]:
def time[R](block: => R): R = {
    val t0 = System.nanoTime()
    val result = block    // call-by-name
    val t1 = System.nanoTime()
    println("Elapsed time: " + (t1 - t0) + "ns")
    result
}

time: [R](block: => R)R


In [7]:
time(logFile.count())

Elapsed time: 1858801078ns


34836

In [8]:
logFile.cache()

/resources/Prezentace/LabData/notebook.log MapPartitionsRDD[5] at textFile at <console>:29

In [9]:
time(logFile.count())

Elapsed time: 3874757477ns


34836

In [ ]:
import org.apache.spark.storage.StorageLevel
val logFile = sc.textFile("/resources/Prezentace/LabData/notebook.log")
logFile.persist(StorageLevel.DISK_ONLY)

In [ ]:
time(logFile.count())

## Key-value pairs RDD - Průměrný věk přátel

Vytvoříme nové RDD ze souboru fakefriends-noheader.csv

In [ ]:
val friends = sc.textFile("/resources/Prezentace/LabData/fakefriends-noheader.csv")

Zobrazíme si prvních pět řádků

In [ ]:
friends.take(5).foreach(println)

Vytvoření RDD které obsahuje tuple (věk,počet přátel)

In [ ]:
val rdd = friends.map(line => {
      val fields = line.split(",")
      val age = fields(2).toInt
      val numFriends = fields(3).toInt
      (age, numFriends)
})
rdd.take(15).foreach(println)

Vytvoříme key-value pair kde klíč bude věk a value bude tuple (celkový počet přátel, počet lidí)

In [ ]:
val totalsByAge = rdd.mapValues(x => (x, 1)).reduceByKey((x,y) => (x._1 + y._1, x._2 + y._2))
totalsByAge.take(5).foreach(println)

Celkový počet přátel vydělíme počtem lidí.

In [ ]:
val averagesByAge = totalsByAge.mapValues(x => x._1 / x._2)

Vypíšeme výsledky.

In [ ]:
val results = averagesByAge.collect()
results.sorted.foreach(println)

# Samostatná práce

Vytvořte RDD ze souboru README.md ve složce LabData.

In [2]:
val readme = sc.textFile("LabData/README.md")

readme = LabData/README.md MapPartitionsRDD[1] at textFile at <console>:27


LabData/README.md MapPartitionsRDD[1] at textFile at <console>:27

Vypiš počet řádku v souboru.

In [3]:
println(readme.count())

98


Vrátit první řádek.

In [7]:
println(readme.first())

# Apache Spark


Vypiš všechny řádky které obsahují slovo Spark

In [9]:
readme.filter(line => line.contains("Spark")).collect().foreach(println)

# Apache Spark
Spark is a fast and general cluster computing system for Big Data. It provides
rich set of higher-level tools including Spark SQL for SQL and DataFrames,
and Spark Streaming for stream processing.
You can find the latest Spark documentation, including a programming
## Building Spark
Spark is built using [Apache Maven](http://maven.apache.org/).
To build Spark and its example programs, run:
["Building Spark"](http://spark.apache.org/docs/latest/building-spark.html).
The easiest way to start using Spark is through the Scala shell:
Spark also comes with several sample programs in the `examples` directory.
    ./bin/run-example SparkPi
    MASTER=spark://host:7077 ./bin/run-example SparkPi
Testing first requires [building Spark](#building-spark). Once Spark is built, tests
Spark uses the Hadoop core library to talk to HDFS and other Hadoop-supported
Hadoop, you must build Spark against the same version that your cluster runs.
for guidance on building a Spark application that